## Group Assignment
### Team Number: XX
### Team Member Names: Derek Tan, Jeff Peng, Yuqian Lin
### Team Strategy Chosen: SAFE

### Abstract

Our portfolio optimization strategy involves the use and implementation of the Modern Portfolio Theory (MPT) and analysis of the Efficient Frontier graphs.

The objective of the portfolio optimization strategy is to maximize the portfolio return while maintaining the minimum portfolio risk. 

Modern Portfolio Theory states that since it is assumed that all investors are risk-adverse, when considering the possible portfolio allocation strategies, the investor will prefer the portfolio that maximizes the possible return while maintaining a given amount of risk.

The Efficient Frontier is a graph that illustrates all possible portfolios portfolio allocation distributions. The x-axis represents the volatility of the portfolio, while the y-axis represents the expected return of the portfolio.

The Efficient Frontier shows the optimized portfolios that offer the highest expected return for a given level of risk and the lowest level of risk for a given level of expected return.

An example of the an Efficient Frontier graph is shown below.

[insert image here]

In [2]:
from IPython.display import display, Math, Latex
from datetime import datetime

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt

In [3]:
# Import Financial Data

tickers = pd.read_csv("Tickers.csv", index_col=False)

start_date = "2018-01-01"
end_date = "2021-10-31"

tickers = [
    "MSFT",
    "AAPL",
    "TWTR",
    "INTC",
    "TSM",
    "GOOG",
    "AMZN",
    "FB",
    "NVDA",
    "TSLA",
    "BB",
]

data = yf.download(tickers, start=start_date, end=end_date)

closing_prices = data["Adj Close"]

closing_prices.head()

[*********************100%***********************]  11 of 11 completed


,AAPL,AMZN,BB,FB,GOOG,INTC,MSFT,NVDA,TSLA,TSM,TWTR
Date,,,,,,,,,,,
2018-01-02,41.188160,1189.010010,12.02,181.419998,1065.000000,42.406261,81.848297,49.386848,64.106003,36.381706,24.510000
2018-01-03,41.180992,1204.199951,13.53,184.669998,1082.479980,40.967072,82.229202,52.637196,63.450001,36.993683,24.450001
2018-01-04,41.372272,1209.589966,13.37,184.330002,1086.400024,40.215797,82.952934,52.914658,62.924000,36.798557,23.990000
2018-01-05,41.843315,1229.140015,13.83,186.850006,1102.229980,40.496399,83.981392,53.363071,63.316002,37.658882,24.320000
2018-01-08,41.687893,1246.869995,14.12,188.279999,1106.939941,40.496399,84.067108,54.998150,67.281998,37.641140,24.590000


To compare the price fluctuations, we will calculate the daily percent change in the price of each stock. This way, it makes the price fluctuations easier to compare, as the price of the stock itself becomes removed from the equation, and we only consider the magnitude of the price movements in comparison to the stock price.

(Yuqian reword this better for me thanks)

In [5]:
# Calculate percent change

percent_change = closing_prices.pct_change().apply(lambda x: np.log(1+x))

percent_change.head()

,AAPL,AMZN,BB,FB,GOOG,INTC,MSFT,NVDA,TSLA,TSM,TWTR
Date,,,,,,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,-0.000174,0.012694,0.118337,0.017756,0.016280,-0.034527,0.004643,0.063739,-0.010286,0.016681,-0.002451
2018-01-04,0.004634,0.004466,-0.011896,-0.001843,0.003615,-0.018509,0.008763,0.005257,-0.008325,-0.005289,-0.018993
2018-01-05,0.011321,0.016033,0.033827,0.013579,0.014466,0.006953,0.012322,0.008439,0.006210,0.023110,0.013662
2018-01-08,-0.003721,0.014322,0.020752,0.007624,0.004264,0.000000,0.001020,0.030181,0.060755,-0.000471,0.011041


We will now analyze the covariance of each stock in relation to one another. The covariance of two stocks is calculated using the following equation:

[insert equation here]

We will store the results of the covariance calculations in 'cov_matrix'.

In [6]:
cov_matrix = percent_change.cov()

cov_matrix

,AAPL,AMZN,BB,FB,GOOG,INTC,MSFT,NVDA,TSLA,TSM,TWTR
AAPL,0.000437,0.000269,0.000321,0.000293,0.000263,0.000287,0.000302,0.000409,0.000365,0.000253,0.000324
AMZN,0.000269,0.000391,0.000239,0.000281,0.000246,0.000222,0.000270,0.000373,0.000319,0.000201,0.000286
BB,0.000321,0.000239,0.001966,0.000320,0.000261,0.000287,0.000274,0.000389,0.000393,0.000255,0.000330
FB,0.000293,0.000281,0.000320,0.000532,0.000293,0.000254,0.000281,0.000395,0.000316,0.000209,0.000394
GOOG,0.000263,0.000246,0.000261,0.000293,0.000345,0.000250,0.000279,0.000356,0.000289,0.000209,0.000309
INTC,0.000287,0.000222,0.000287,0.000254,0.000250,0.000593,0.000291,0.000426,0.000340,0.000273,0.000328
MSFT,0.000302,0.000270,0.000274,0.000281,0.000279,0.000291,0.000361,0.000406,0.000346,0.000235,0.000309
NVDA,0.000409,0.000373,0.000389,0.000395,0.000356,0.000426,0.000406,0.000925,0.000526,0.000405,0.000468
TSLA,0.000365,0.000319,0.000393,0.000316,0.000289,0.000340,0.000346,0.000526,0.001670,0.000324,0.000409
TSM,0.000253,0.000201,0.000255,0.000209,0.000209,0.000273,0.000235,0.000405,0.000324,0.000453,0.000252


We will now analyze the correlation of each stock in relation to one another. To do this, we will use a correlation matrix.

The correlation of two stocks is calculated using the following equation:

[insert equation here]

Note that each stock has a correlation of 1 with itself, a perfect positive correlation.

We will store the results of the correlation calculations in 'corr_matrix'.

In [7]:
corr_matrix = percent_change.corr()

corr_matrix

,AAPL,AMZN,BB,FB,GOOG,INTC,MSFT,NVDA,TSLA,TSM,TWTR
AAPL,1.000000,0.650743,0.346371,0.607171,0.678099,0.564265,0.760841,0.642725,0.427233,0.568665,0.459454
AMZN,0.650743,1.000000,0.272672,0.615939,0.670549,0.461625,0.717986,0.620040,0.394249,0.478222,0.429286
BB,0.346371,0.272672,1.000000,0.312906,0.316541,0.265822,0.324881,0.288105,0.216904,0.270459,0.220331
FB,0.607171,0.615939,0.312906,1.000000,0.683675,0.452693,0.640478,0.562783,0.335484,0.425825,0.506402
GOOG,0.678099,0.670549,0.316541,0.683675,1.000000,0.551392,0.791742,0.630835,0.380510,0.527938,0.493828
INTC,0.564265,0.461625,0.265822,0.452693,0.551392,1.000000,0.628606,0.574492,0.341996,0.527175,0.398724
MSFT,0.760841,0.717986,0.324881,0.640478,0.791742,0.628606,1.000000,0.703592,0.445289,0.582612,0.481585
NVDA,0.642725,0.620040,0.288105,0.562783,0.630835,0.574492,0.703592,1.000000,0.422796,0.625859,0.456286
TSLA,0.427233,0.394249,0.216904,0.335484,0.380510,0.341996,0.445289,0.422796,1.000000,0.372692,0.296944
TSM,0.568665,0.478222,0.270459,0.425825,0.527938,0.527175,0.582612,0.625859,0.372692,1.000000,0.350705


In [8]:
# WIP

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Derek, Yuqian, Jeff